In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input \data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/kaggle/input/medical-insurance-premium-prediction/Medicalpremium.csv')
df.head()

## Knowing about data & target Varible. 

1. All features 
2. Target Variable 
3. Primary correlations using heatmap(sns)


In [ ]:
df.info()


`Above data has no value misssing & all values are in integer. 


In [ ]:
df.columns

## Target Feature

In [ ]:
## Target Variable 
sns.kdeplot(df.PremiumPrice)

In [ ]:
for i in df.columns:
    print(i, len(df[i].unique()))

In [ ]:
binary_feature = [i for i in df.columns if (len(df[i].unique()) == 2)]
target_feature = 'PremiumPrice'
disceate_feature = [i for i in df.columns if (i not in binary_feature) and (i not in target_feature)]

### Working on Binary features 

In [ ]:
fig, ax = plt.subplots(5,2, figsize = (20,30))
sns.countplot(ax =ax[0,0], data = df , x = binary_feature[0])
sns.barplot(ax =ax[0,1], data = df , x = binary_feature[0], y = target_feature)
sns.countplot(ax =ax[1,0], data = df , x = binary_feature[1])
sns.barplot(ax =ax[1,1], data = df , x = binary_feature[1], y = target_feature)
sns.countplot(ax =ax[2,0], data = df , x = binary_feature[2])
sns.barplot(ax =ax[2,1], data = df , x = binary_feature[2], y = target_feature)
sns.countplot(ax =ax[3,0], data = df , x = binary_feature[3])
sns.barplot(ax =ax[3,1], data = df , x = binary_feature[3], y = target_feature)
sns.countplot(ax =ax[4,0], data = df , x = binary_feature[4])
sns.barplot(ax =ax[4,1], data = df , x = binary_feature[4], y = target_feature)
# sns.countplot(ax =ax[5,0], data = df , x = binary_feature[5])//
# sns.barplot(ax =ax[5,1], data = df , x = binary_feature[5], y = target_feature)

Based on visualzation following point may be colcluded.
1. If you have a disease the premium price is higher than others
2. count of Blood pressure & diabities patient is higher than other list of disease

In [ ]:
disceate_feature

In [ ]:
fig, ax = plt.subplots(4,2, figsize = (20,15))
sns.countplot(ax =ax[0,0], data = df , x = disceate_feature[0])
sns.barplot(ax =ax[0,1], data = df , x = disceate_feature[0], y = target_feature)
sns.countplot(ax =ax[1,0], data = df , x = disceate_feature[1])
sns.barplot(ax =ax[1,1], data = df , x = disceate_feature[1], y = target_feature)
sns.countplot(ax =ax[2,0], data = df , x = disceate_feature[2])
sns.barplot(ax =ax[2,1], data = df , x = disceate_feature[2], y = target_feature)
sns.countplot(ax =ax[2,0], data = df , x = disceate_feature[2])
sns.barplot(ax =ax[2,1], data = df , x = disceate_feature[2], y = target_feature)
sns.countplot(ax =ax[3,0], data = df , x = disceate_feature[3])
sns.barplot(ax =ax[3,1], data = df , x = disceate_feature[3], y = target_feature)

Above graph shows following basic conclusions:
1. As age & number of surgeries increases the premiunprice also increase.
2. we don't conclude relationship of Height or weight to the target variable. 

## checking for correlations

In [ ]:
plt.figure(figsize =(20,8))
sns.heatmap(df.corr(), annot =True)

## Deriving new features 

In [ ]:
df['bmi'] = (df.Weight)/(((df.Height)/100)**2)

In [ ]:
len(df.columns)

In [ ]:
fig, ax = plt.subplots(3,2, figsize = (20,15))
sns.boxplot(ax =ax[0,0], data = df , x = 'Age')
sns.boxplot(ax =ax[0,1], data = df , x = "Height")
sns.boxplot(ax =ax[1,0], data = df , x = "Weight")
sns.boxplot(ax =ax[1,1], data = df , x = "NumberOfMajorSurgeries")
sns.boxplot(ax =ax[2,0], data = df , x = "PremiumPrice")
sns.boxplot(ax =ax[2,1], data = df , x = 'bmi')
cols = [i for i  in df.columns if i not in binary_feature]
cols

We have sitution related to outlier . because of having less amount of data we can not drop samples.

## Model building

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

X = df.drop('PremiumPrice', axis =1)
y =df.PremiumPrice

In [ ]:
model = SelectFromModel(Lasso(alpha = 0.05)).fit(X,y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_scaled = scaler.transform(X)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
scores = []

for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    scores.append(model.score(X_test, y_test))

print(np.mean(scores))

In [ ]:
scores = []
# on scaled data
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    scores.append(model.score(X_test, y_test))

print(np.mean(scores))

In [ ]:
scores = []
from sklearn.ensemble import RandomForestRegressor
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2)
    
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    scores.append(model.score(X_test, y_test))

print(np.mean(scores))

In [ ]:
import numpy as np

target= np.array(df['PremiumPrice'])
features = df.drop('PremiumPrice', axis = 1)
feature_list = list(features.columns)
features = np.array(features)

## RANDOM FOREST - KFOLD AND MODEL 


from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
    
kf = KFold(n_splits=10,random_state=42,shuffle=True)
accuracies = []
for train_index, test_index in kf.split(features):

    data_train   = features[train_index]
    target_train = target[train_index]

    data_test    = features[test_index]
    target_test  = target[test_index]

    rf = RandomForestRegressor(n_estimators = 10, 
                               random_state = 42, 
                               criterion = 'mse',
                               bootstrap=True)
    
    rf.fit(data_train, target_train)

    predictions = rf.predict(data_test)

    errors = abs(predictions - target_test)

    print('Mean Absolute Error:', round(np.mean(errors), 2))
    
    mape = 100 * (errors / target_test)
    accuracy = 100 - np.mean(mape)
    print('Accuracy:', round(accuracy, 2), '%.')
    print('train  accuracy  ', np.sqrt(mean_squared_error(target_train, rf.predict(data_train))))
    print('train  accuracy  ', np.sqrt(mean_squared_error(target_test, rf.predict(data_test))))
    print('r2score train ', r2_score(target_train, rf.predict(data_train)))
    print('r2score test ' , r2_score(target_test, rf.predict(data_test)))
    
    

    accuracies.append(accuracy)

average_accuracy = np.mean(accuracies)
print('Average accuracy:', average_accuracy)